In [ ]:
import sys

from modules.utils import CategoricalCrossentropyLoss, KLDivergenceLoss, MSELoss


sys.path.append("../../")
from agents.random import RandomAgent
from hyperparameter_optimization.hyperopt import (
    marl_objective,
    set_marl_config,
    MarlHyperoptConfig,
)
from hyperopt import atpe, tpe, fmin, space_eval
from hyperopt.exceptions import AllTrialsFailed

from muzero.muzero_agent_torch import MuZeroAgent
from agent_configs import MuZeroConfig
from game_configs import CartPoleConfig, TicTacToeConfig
from agents.tictactoe_expert import TicTacToeBestAgent
from muzero.action_functions import action_as_onehot, action_as_plane
from torch.optim import SGD, Adam

env = CartPoleConfig().make_env()
# env = FrameStackWrapper(env, 4)
# TODO: FRAME STACKING FOR CARTPOLE IF POSSIBLE
# TODO: ADD NORMALIZE INPUT TO 0 AND 1 LAYER
params = {
    "num_simulations": 25,  # 25
    "per_alpha": 0.0,  # 0.5-1.0, 0.6
    "per_beta": 0.0,  # 1.0
    "per_beta_final": 0.0,  # 1.0
    "root_dirichlet_alpha": 0.5,  # 0.5 - 1.0, 0.3
    "dense_layer_widths": [
        64,
        8,
    ],  # 1-3 layers, 64-128, with a dynamics size of 8-16
    "dynamics_dense_layer_widths": [
        64,
        8,
    ],  # 1-3 layers, 64-128, with a dynamics size of 8-16
    "residual_layers": [],
    "reward_dense_layer_widths": [],
    "reward_conv_layers": [],
    "actor_dense_layer_widths": [],
    "actor_conv_layers": [],
    "critic_dense_layer_widths": [],
    "critic_conv_layers": [],
    "to_play_dense_layer_widths": [],
    "to_play_conv_layers": [],
    "known_bounds": [1, 500],
    "support_range": 31,  # 601 but only for value and not reward, 300
    "minibatch_size": 64,  # 64-128
    "replay_buffer_size": 1000,  # 10k -20k
    "min_replay_buffer_size": 500,
    "gumbel": True,
    "gumbel_m": 2,
    "policy_loss_function": KLDivergenceLoss(),  # KLDivergenceLoss()
    "reward_loss_function": CategoricalCrossentropyLoss(),  # CategoricalCrossentropyLoss(),
    "value_loss_function": CategoricalCrossentropyLoss(),  # CategoricalCrossentropyLoss(),
    "training_steps": 20000,
    "transfer_interval": 1,  # ?
    "num_workers": 4,
    "discount_factor": 0.999,  # 0.99 too low? but good for alpha zero? 0.997 - 0.999
    "unroll_steps": 5,  # 5
    "n_step": 50,  # 5, 10, 50 or 500 (50 or 500 recommended)
    # "temperatures": [3, 2, 1, 0.5, 0.25, 0.125, 0.075, 0.01, 0.0],
    # "temperature_updates": [100, 200, 300, 400, 500, 600, 700, 800],
    # "temperature_with_training_steps": True,
    "temperatures": [1.0, 0.0],
    "temperature_updates": [15],
    "temperature_with_training_steps": False,
    "learning_rate": 0.001,
    "optimizer": Adam,
    "value_loss_factor": 0.25,  # 0.25
    "reanalyze_ratio": 0.0,  # 100% should be reanalyzed (how?)
    "injection_frac": 0.0,
    "value_prefix": False,
    "consistency_loss_factor": 0.0,
    # TODO: REWARD SUPPORT SIZE DIFFERENT THAN VALUE SUPPORT SIZE (either scalar or support range 1 (2 buckets))
    # c_puct 1.0 - 1.5
}
game_config = CartPoleConfig()
config = MuZeroConfig(config_dict=params, game_config=game_config)

agent = MuZeroAgent(
    env=env,
    config=config,
    name="cartpole_test_7",
    device="cpu",
    test_agents=[],
)
agent.checkpoint_interval = 100
agent.test_interval = 1000
agent.test_trials = 10

agent.train()

Using default save_intermediate_weights     : False
Using         training_steps                : 20000
Using default adam_epsilon                  : 1e-08
Using default momentum                      : 0.9
Using         learning_rate                 : 0.001
Using default clipnorm                      : 0
Using         optimizer                     : <class 'torch.optim.adam.Adam'>
Using default weight_decay                  : 0.0
Using default loss_function                 : <class 'utils.utils.MSELoss'>
Using default activation                    : relu
Using         kernel_initializer            : None
Using         minibatch_size                : 64
Using         replay_buffer_size            : 1000
Using         min_replay_buffer_size        : 500
Using default num_minibatches               : 1
Using default training_iterations           : 1
Using default print_interval                : 100
Using         known_bounds                  : [1, 500]
Using         residual_layers        

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/homebrew/Cellar/python@3.10/3.10.14/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/opt/homebrew/Cellar/python@3.10/3.10.14/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
  File "/Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/experiments/tictactoe_muzero_nfsp/../../muzero/muzero_agent_torch.py", line 6, in <module>
    from packages.utils.utils.utils import KLDivergenceLoss
ImportError: cannot import name 'KLDivergenceLoss' from 'packages.utils.utils.utils' (/Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/experiments/tictactoe_muzero_nfsp/../../packages/utils/utils/utils.py)


learned
learned
learned
learned
learned
learned
learned
learned
learned
learned
learned
learned
learned
learned
learned
score:  500.0
learned
learned
learned
learned
learned
learned
learned
learned
learned
learned
learned
learned
learned
learned
learned
learned
learned
learned
learned
learned
learned
learned
learned
learned
learned
learned
learned
learned
learned


Process Process-11:
Process Process-1:
Process Process-2:
Process Process-3:
Process Process-4:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.10/3.10.14/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/homebrew/Cellar/python@3.10/3.10.14/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/experiments/tictactoe_muzero_nfsp/../../muzero/muzero_agent_torch.py", line 308, in worker_fn
    score, num_steps = self.play_game(
  File "/Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/experiments/tictactoe_muzero_nfsp/../../muzero/muzero_agent_torch.py", line 1644, in play_game
    prediction = self.predict(
  File "/Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/exper

KeyboardInterrupt: 

In [ ]:
import sys

import numpy as np

sys.path.append("../../")

from hyperparameter_optimization.hyperopt import save_search_space


import dill as pickle
from hyperopt import hp
from hyperopt.pyll import scope
from utils import CategoricalCrossentropyLoss, MSELoss, generate_layer_widths
import gymnasium as gym
import torch
from muzero.action_functions import action_as_onehot as action_function
from torch.optim import Adam, SGD

search_space = {
    "kernel_initializer": hp.choice(
        "kernel_initializer",
        [
            "he_uniform",
            "he_normal",
            "glorot_uniform",
            "glorot_normal",
            "orthogonal",
        ],
    ),
    "known_bounds": hp.choice("known_bounds", [[0, 500]]),
    "residual_filters": hp.choice("residual_filters", [[]]),
    "residual_stacks": hp.choice("residual_stacks", [[]]),
    "conv_layers": hp.choice("conv_layers", [[]]),
    "actor_and_critic_conv_filters": hp.choice("actor_and_critic_conv_filters", [[]]),
    "reward_conv_layers": hp.choice("reward_conv_layers", [[]]),
    "actor_dense_layer_widths": hp.choice("actor_dense_layer_widths", [[32], []]),
    "critic_dense_layer_widths": hp.choice("critic_dense_layer_widths", [[32], []]),
    "reward_dense_layer_widths": hp.choice("reward_dense_layer_widths", [[32], []]),
    "to_play_dense_layer_widths": hp.choice("to_play_dense_layer_widths", [[]]),
    "dense_layer_widths": hp.choice(
        "dense_layer_widths", [[32], [32, 32], [32, 32, 32]]
    ),
    "noisy_sigma": hp.choice("noisy_sigma", [0.0]),
    "value_loss_factor": hp.choice("value_loss_factor", [1.0]),
    "root_dirichlet_alpha": hp.quniform(
        "root_dirichlet_alpha", 0.1, 1.0, 0.1
    ),  # hp.choice("root_dirichlet_alpha", [0.3, 1.0, 2.0]),
    "root_exploration_fraction": hp.choice("root_exploration_fraction", [0.25]),
    "num_simulations": scope.int(
        hp.qloguniform("num_simulations", np.log(25), np.log(150), 25)
    ),
    # "temperature_updates": hp.choice("temperature_updates", [[15000, 30000]]),
    "temperatures": hp.choice("temperatures", [[1.0, 0.5, 0.25]]),
    "temperature_with_training_steps": hp.choice(
        "temperature_with_training_steps", [True]
    ),
    "clip_low_prob": hp.choice("clip_low_prob", [0.0]),
    "pb_c_base": hp.choice("pb_c_base", [19652]),
    "pb_c_init": hp.choice("pb_c_init", [1.25]),
    "value_loss_function": hp.choice(
        "value_loss_function", [CategoricalCrossentropyLoss()]
    ),
    "reward_loss_function": hp.choice(
        "reward_loss_function", [CategoricalCrossentropyLoss()]
    ),
    "policy_loss_function": hp.choice(
        "policy_loss_function", [CategoricalCrossentropyLoss()]
    ),
    "training_steps": scope.int(hp.quniform("training_steps", 10000, 10100, 1000)),
    "minibatch_size": scope.int(2 ** (hp.quniform("minibatch_size", 3, 7, 1))),
    "min_replay_buffer_size": scope.int(
        hp.qloguniform("min_replay_buffer_size", np.log(1000), np.log(1100), 1000)
    ),
    "replay_buffer_size": scope.int(10 ** (hp.quniform("replay_buffer_size", 3, 6, 1))),
    "unroll_steps": hp.choice("unroll_steps", [5]),
    "n_step": hp.choice("n_step", [10, 500]),
    "clipnorm": hp.choice(
        "clipnorm",
        [0.0, scope.int(10 ** (hp.quniform("clip_val", 0, 2, 1)))],
        # "clipnorm",
        # [0.0],
    ),
    "weight_decay": hp.choice("weight_decay", [1e-4]),
    "per_alpha": hp.choice("per_alpha", [0.0]),
    "per_beta": hp.choice("per_beta", [0.0]),
    "per_beta_final": hp.choice("per_beta_final", [0.0]),
    "per_epsilon": hp.choice("per_epsilon", [1e-4]),
    "action_function": hp.choice("action_function", [action_function]),
    "multi_process": hp.choice(
        "multi_process",
        [
            {
                "multi_process": True,
                "num_workers": scope.int(hp.quniform("num_workers", 1, 5, 1)),
            },
            # {
            #     "multi_process": False,
            #     "games_per_generation": scope.int(
            #         hp.qloguniform("games_per_generation", np.log(8), np.log(32), 8)
            #     ),
            # },
        ],
    ),
    "lr_ratio": hp.choice("lr_ratio", [float("inf")]),
    "support_range": scope.int(hp.quniform("support_range", 601, 601 + 1e-8, 1)),
}

initial_best_config = []

search_space, initial_best_config = save_search_space(search_space, initial_best_config)

In [ ]:
def prep_params(params):
    params["residual_layers"] = []
    params["actor_conv_layers"] = []
    params["critic_conv_layers"] = []
    params["to_play_conv_layers"] = []

    if params["multi_process"]["multi_process"] == True:
        params["num_workers"] = params["multi_process"]["num_workers"]
        params["multi_process"] = True
    else:
        params["games_per_generation"] = params["multi_process"]["games_per_generation"]
        params["multi_process"] = False

    # if params["optimizer"]["optimizer"] == "adam":
    #     params["adam_epsilon"] = params["optimizer"]["adam_epsilon"]
    #     params["optimizer"] = Adam
    # elif params["optimizer"]["optimizer"] == "sgd":
    #     params["momentum"] = params["optimizer"]["momentum"]
    #     params["optimizer"] = SGD
    params["temperature_updates"] = [
        params["training_steps"] / 2,
        3 * params["training_steps"] / 4,
    ]
    return params

In [ ]:
from agents.random import RandomAgent
from hyperparameter_optimization.hyperopt import (
    sarl_objective,
    set_sarl_config,
    SarlHyperoptConfig,
)
from hyperopt import atpe, tpe, fmin, space_eval
from hyperopt.exceptions import AllTrialsFailed

from muzero.muzero_agent_torch import MuZeroAgent
from agent_configs import MuZeroConfig
from game_configs import CartPoleConfig

search_space_path, initial_best_config_path = (
    "search_space.pkl",
    "best_config.pkl",
)
# search_space = pickle.load(open(search_space_path, "rb"))
# initial_best_config = pickle.load(open(initial_best_config_path, "rb"))
file_name = "cartpole_muzero"
max_trials = 128
trials_step = 128  # how many additional trials to do after loading the last ones

set_sarl_config(
    SarlHyperoptConfig(
        file_name=file_name,
        eval_method="rolling_average",
        make_env=CartPoleConfig().make_env,
        prep_params=prep_params,
        agent_class=MuZeroAgent,
        agent_config=MuZeroConfig,
        game_config=CartPoleConfig,
        checkpoint_interval=50,
        test_interval=100,
        test_trials=25,
        last_n_rolling_avg=3,
        device="cpu",
    )
)

try:  # try to load an already saved trials object, and increase the max
    trials = pickle.load(open(f"./{file_name}_trials.p", "rb"))
    print("Found saved Trials! Loading...")
    max_trials = len(trials.trials) + trials_step
    print(
        "Rerunning from {} trials to {} (+{}) trials".format(
            len(trials.trials), max_trials, trials_step
        )
    )
except:  # create a new trials object and start searching
    print("No saved Trials! Starting from scratch.")
    trials = None

best = fmin(
    fn=sarl_objective,  # Objective Function to optimize
    space=search_space,  # Hyperparameter's Search Space
    algo=atpe.suggest,  # Optimization algorithm (representative TPE)
    max_evals=max_trials,  # Number of optimization attempts
    trials=trials,  # Record the results
    # early_stop_fn=no_progress_loss(5, 1),
    trials_save_file=f"./{file_name}_trials.p",
    points_to_evaluate=initial_best_config,
    show_progressbar=False,
)
print(best)
best_trial = space_eval(search_space, best)
# gc.collect()